In [7]:
import pandas as pd
import spacy
import en_core_web_sm
from collections import Counter

In [15]:
df = pd.read_csv("./downloads/reviews100.csv")
df

,stars,comment
0,5.0 out of 5 stars,Quiet machine with ample capacity both in numb...
1,1.0 out of 5 stars,Very disappointed. Received shredder and plugg...
2,1.0 out of 5 stars,"This shredder has only lasted 5 months, my las..."
3,2.0 out of 5 stars,There isn’t much that can go wrong with these ...
4,3.0 out of 5 stars,I give this shredder three and a half stars. ...
...,...,...
995,4.0 out of 5 stars,This shredder works for basic bills and things...
996,5.0 out of 5 stars,"Shredders have been the bane of my existence, ..."
997,5.0 out of 5 stars,I needed a shredder for the house - when you h...
998,5.0 out of 5 stars,My 6-year old Staples shredder died after LOTS...


In [11]:
df['comment'] = df['comment'].apply(lambda x: x.strip())
df = df.sort_values(['stars'], ascending=False).iloc[0:30] # get 30 top reviews 
df

,stars,comment
25,5.0 out of 5 stars,I shred junk mail like it's a part time job. S...
21,5.0 out of 5 stars,Quiet machine with ample capacity both in numb...
88,5.0 out of 5 stars,Whether you’re a college student or a professi...
53,5.0 out of 5 stars,I hesitated paying up for this shedder but aft...
87,5.0 out of 5 stars,Reviewing the 12-sheet Amazon Basics paper shr...
27,5.0 out of 5 stars,"Update: After over a year, the shredder is sti..."
82,5.0 out of 5 stars,The Shredder is listed as a light/medium duty ...
32,5.0 out of 5 stars,"I""m so glad I got this paper shredder. I'd ha..."
33,5.0 out of 5 stars,AmazonBasics are generally good quality items....
78,5.0 out of 5 stars,If I could give this six stars I would. What a...


In [12]:
# nlp = spacy.load("en_core_web_sm") # buggy on ipynb
nlp = en_core_web_sm.load() # for ipynb
noun_adj_pairs = {}
# print(df['comment'])
for comment in enumerate(df['comment']):
    # print(comment[1])
    doc = nlp(str(comment[1])) # change tuple to str for nlp
    doc_len = len(doc)
    for i, token in enumerate(doc): #token has TEXT, LEMMA, POS, TAG, DEP, SHAPE, ALPHA, STOP
        if token.pos_ not in ('NOUN', 'PROPN'):
            continue
        for j in range(i+1, doc_len): # check for pairs after identified noun/pnoun
            if doc[j].pos_ == 'ADJ':
                pair = (str(token), str(doc[j])) # change to str. ele in dict keys has to be hashable even inside tuples
                # check if already exist in dictionary
                if pair in noun_adj_pairs:
                    noun_adj_pairs[pair] += 1
                else:
                    noun_adj_pairs[pair] = 1
top_10_noun_adj_pairs = dict(Counter(noun_adj_pairs).most_common(5))
top_10_noun_adj_pairs
# top_10_noun_adj_pairs = {key: val for key, val in sorted(noun_adj_pairs.items(), key=lambda item: item[1])}
# print(noun_adj_pairs.keys())
# noun_adj_pairs = dict(sorted(noun_adj_pairs.items()))
# print(noun_adj_pairs)

{('shredder', 'small'): 36,
 ('shredder', 'little'): 34,
 ('shredder', 'few'): 33,
 ('shredder', 'sure'): 21,
 ('shredder', 'full'): 21}